In [39]:
import DSL._, Constants._

import DSL._, Constants._

De Bruijn indices are numbers that disambiguate each shadowed variable inside lambdas.

For example:

     λ(x : Natural) → λ(x : Natural) → 123 + x

This code shadows the outer `x` inside the inner lambda. An equivalent code without shadowing is:

     λ(x : Natural) → λ(t : Natural) → 123 + t

Here we just renamed the inner `x` to `t`. What if we do not want to rename `x` to `t` but still want to refer to the outer `x`?
In µDhall, we can write `x@1` for that:

     λ(x : Natural) → λ(x : Natural) → 123 + x@1

This code is equivalent to:

     λ(x : Natural) → λ(t : Natural) → 123 + x

The `1` in `x@1` is the de Bruijn index of the outer variable `x` when accessed from the inner scope.
De Bruijn indices are non-negative integers.

We usually do not write `x@0`, we write just `x`.

Each de Bruijn index points to a specific nested lambda in some outer scope for a variable with a given name. For example, in this code:

     λ(x : Natural) → λ(t : Natural) → λ(x : Natural) → 123 + x@1

the variable `x@1` still points to the outer `x`. The presence of another lambda with the argument `t` does not matter for counting the nested depth for `x`.

At top level, it is invalid to use an index that is greater than the number of nested lambdas. For example, these expressions are invalid at top level (as there cannot be any outer scope):

     λ(x : Natural) → 123 + x@1
     λ(x : Natural) → λ(x : Natural) → 123 + x@2

At top level, these expressions are just as invalid as the expression `123 + x` since we never defined `x`.

The variable `x` in the expression `123 + x` is considered **free**; meaning that it should be defined in the outer scope.
Similarly, `x@1` in `λ(x : Natural) → 123 + x@1` is free. It is invalid to have expressions with free variables at top level.
At the top level, all variables must be bound. Expressions with free variables must be within bodies of some functions that bind their free variables.

When a function of the form `λ x → ...` is applied to an argument, we will need to substitute the outer `x`. Then we may need to recalculate some de Bruijn indices.

Suppose we would like to evaluate a function applied to an argument in this code:

     ( (x : Natural) → λ(y : Natural) → λ(x : Natural) → x + x@1 + x@2 ) y

This expression has free variables `y` and `x@2`, so it can occur only within a function body that binds `x` and `y`. It is worth remarking that here we are about to evaluate an expression "under a lambda". That is, we are going to simplify the body of a function before applying that function.

To evaluate this expression correctly, we cannot simply substitute `y` instead of `x`. Instead:

- the outer `x` corresponds to `x@1` within the expression, so we need to substitute `y` instead of `x@1` while keeping `x` and `x@2` unchanged
- `y` is already bound in the inner scope; so, we need to write `y@1` instead of `y`, in order to refer to the free variable `y` in the outside scope
- as we remove the outer `x`, the free variable `x@2` will now become `x@1`.

So, we must evaluate the given expression like this:

     ( (x : Natural) → λ(y : Natural) → λ(x : Natural) → x + x@1 + x@2 ) y
       = 
     λ(y : Natural) → λ(x : Natural) → x + y@1 + x@1  

During a single substitution, sometimes we need to shift the indices upwards by 1, and sometimes downwards by 1. In order to be able to evaluate expressions and substitute arbitrary variables with de Bruijn indices correctly, we will now implement the general "shift" operation. This operation is defined in the Dhall standard [here](https://github.com/dhall-lang/dhall-lang/blob/master/standard/shift.md) as a function of _four_ arguments:

     // d = +1 or d = -1
     // x is the variable symbol
     // m is the minimum index for shifting; we will shift x@n only if n >= m
     // e1 is an expression where we need to shift
     
     shift(d, x, m, e1) = e2

The Dhall documentation uses the "proof notation" for all definitions. The proof notation shows how to prove a desired statement (or "judgment") by proving some other statements first, or by referring to axioms. The equation `shift(d, x, m, e1) = e2` is seen as a judgment that the result of evaluating the `shift` function with some arguments happens to equal `e2`. But we prefer to interpret that equation as a recipe for computing `e2` given `d`, `x`, `m`, and `e1`.

Adapting the Dhall specification to µDhall, we find the following definition of `shift`:

     shift(d, x, m, x@n) = x@(n + d)  // if n >= m

res42: String = "v@1"